This script runs EC821B HW03 Pt2: simulation of a multivariate normal probability.

Consider some binary random variable $D_i$, where 
$\Pr(D_i = 1\vert X_i) = G(X_i;\ \theta_0)$, some parametric density.

Suppose we believe there is a latent variable, $y_i^{*} = X_i^{\prime}\beta_0 + \epsilon_i$, such that $D_i = 1 \iff y_i^{*} \ge 0 \iff X_i^{\prime}\beta_0 \ge -\epsilon_i$.  
If we further assume $\epsilon_i|_{X_i} \sim \mathsf{N}(0,\sigma^2)$, then $G(X_i;\ \theta_0) = \Phi\left(\frac{X_i^{\prime}\beta_0}{\sigma}\right) = \Phi(X_i^{\prime}\gamma_0)$.



In [1]:
# Set Working Directory
cd("/Volumes/HDD_lxw/Users/LukeXWatson/Dropbox/02_CLASSES/2017_02_Spring/821B_Cross Section/HW/HW3")
# Get Standard Packages Running
using Distributions, Plots, DataFrames, StatsFuns, Optim

In [2]:
############ DATA IMPORT
# Imported Rows Come as DataFrames
age = readtable("CPS87/age.dat", separator = ' ', header = false);
edu = readtable("CPS87/educ.dat", separator = ' ', header = false);
race = readtable("CPS87/race.dat", separator = ' ', header = false);
msa = readtable("CPS87/smsa.dat", separator = ' ', header = false);
wks = readtable("CPS87/wkwrkly.dat", separator = ' ', header = false);
############ DATA FORMAT
# Put into a single df
data = [age edu race msa wks];
# Change column names
names!(data, [:age, :edu, :race, :msa, :wks]);
# Create Constant
data[:cons] = 1;
# Create Full-time variable
data[:full] = 0;
data[(data[:wks].>=40),:full] = 1;
# Create Experience Variable
data[:expr] = data[:age] - data[:edu] - 6;
# Create White dummy
data[:wht] = 0;
data[(data[:race].==1),:wht] = 1;
# Correct MSA dummy (change 2 to 0)
data[(data[:msa].==2),:msa] = 0;
##### Divide edu and expr by mean
data[:eduM] = data[:edu] ./ mean(data[:edu])
data[:exprM] = data[:expr] ./ mean(data[:expr])
#writetable("data87.csv", data)
#data[1:5,:]
# Separate into data for the estimation:
X = data[:,[:cons, :eduM, :wht, :msa, :exprM]];
F = data[:,[:full]];
N = nrow(X);
k = ncol(X);
# Turn into float values for faster calculations 
# NOTE: Mixing types (eg real numbers and integers) increases computational time by factor of 10 (or more!)
X = Array{Float64}(Any(X));
F = Array{Float64}(Any(F));
############ DATA PREVIEW
X[1:5,:]

5×5 Array{Float64,2}:
 1.0  0.575653  1.0  1.0  2.48053 
 1.0  1.48025   1.0  1.0  0.563757
 1.0  0.904598  1.0  1.0  1.24026 
 1.0  0.986834  1.0  1.0  0.732883
 1.0  1.48025   1.0  1.0  0.789259

In [3]:
############ Initial Values
# OLS Initial Values
gmOLS = (inv(X'*X)*(X'*F))[1:5];
# Included are the STATA results for comparison
gmSTATA = [-1.88, 1.78, 0.27, 0.05, 0.29];
# Set Options for Optimizer
opts = Optim.Options(f_tol = 1e-20, g_tol = 1e-8, iterations = 1000);

Under the assumptions given, we have a standard Probit model.  
Gathering items, we have: 

$\hat{\gamma}_{MLE} = \arg\min\{Q_n(\gamma)\}$

$Q_n(\gamma) = \frac{-1}{N}\sum_{i\in N} d_i\ln[\Phi(X_i^{\prime}\gamma)] + (1 - d_i)\ln[1 - \Phi(X_i^{\prime}\gamma)] = \frac{-1}{N}\sum_{i\in N} l(X_i,\gamma)$ 

In [4]:
############ Construct Likelihood Function
# Set Distribution: Standard Normal
function f_lnLike(gm::Vector)
    ## This creates a lnL function of given parameters for MLE
    # (Re)Set Random Seed to Prevent "Chattering" - use same simulations each iteration
    srand(537099)
    # Preallocate Vectors
    t1 = Array(Float64,1,k)
    t2 = Array(Float64,N,1)
    t3 = Array(Float64,N,1)
    t4 = Array(Float64,N,1)
    t5 = Array(Float64,1,1)
    # Calculations
    t1 = [gm[1] gm[2] gm[3] gm[4] gm[5]]
    t2 = X*t1'
    # CDF of Standard Normal dist. eval. at each obs.
    t3 = cdf(Normal(), t2)
    # Column vector of lnL_i
    t4 = F.*log(t3) + (1 - F).*log(1-t3)
    # Sum of lnL_i = lnL; return the negation for minimization
    t5 = sum(t4)
    return (-1)*t5
end

f_lnLike (generic function with 1 method)

In [5]:
# See if function works and eval. against STATA
print("STATA Check: $(f_lnLike(gmSTATA)) and OLS Trial $(f_lnLike(gmOLS))")

STATA Check: 28275.31692479358 and OLS Trial 30785.888430774758

In [6]:
# This used the NM optimization routine to search for the gamma that minimizes(-lnLk)
MLE = optimize(f_lnLike, gmOLS, opts)

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-0.09421108407456535,0.568001095931335, ...]
 * Minimizer: [-1.8824966206978404,1.7791887589036641, ...]
 * Minimum: 2.827517e+04
 * Iterations: 292
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 372

In [7]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
Julia = round(Optim.minimizer(MLE),3), 
STATA = round(gmSTATA,3),
OLS = round(gmOLS,3))

,vars,Julia,STATA,OLS
1,cons,-1.882,-1.88,-0.094
2,edu,1.779,1.78,0.568
3,wht,0.273,0.27,0.092
4,msa,0.053,0.05,0.016
5,expr,0.291,0.29,0.099


Now for the Asymptotic Variance Estimator:  

$\mathsf{AVar}(\gamma_0) = \mathsf{E}[\frac{\partial}{\partial\gamma\partial\gamma^{\prime}} l(X_i,\gamma_0)]^{-1}\cdot 
\mathsf{E}[\frac{\partial}{\partial\gamma}\mathcal{l}(X_i,\gamma)\frac{\partial}{\partial\gamma^{\prime}} l(X_i,\gamma_0)]\cdot
\mathsf{E}[\frac{\partial}{\partial\gamma\partial\gamma^{\prime}} l(X_i,\gamma_0)]^{-1}$

$\widehat{\mathsf{AVar}}\left(\sqrt{N}(\hat{\gamma} - \gamma_o)\right) = 
[\frac{1}{N}\sum_{i\in N}\frac{\partial}{\partial\gamma\partial\gamma^{\prime}} l(X_i,\hat{\gamma})]^{-1}\cdot 
[\frac{1}{N}\sum_{i\in N}\frac{\partial}{\partial\gamma}\mathcal{l}(X_i,\gamma)\frac{\partial}{\partial\gamma^{\prime}} l(X_i,\hat{\gamma})]\cdot
[\frac{1}{N}\sum_{i\in N}\frac{\partial}{\partial\gamma\partial\gamma^{\prime}} l(X_i,\hat{\gamma})]^{-1}$

$\frac{\partial}{\partial\gamma} l(X_i,\hat{\gamma}) = \frac{D_i - \Phi(X_i^{\prime}\gamma)}{\Phi(X_i^{\prime}\gamma)\cdot[1-\Phi(X_i^{\prime}\gamma)]}\phi(X_i^{\prime}\gamma)\cdot X_i^{\prime}$

$\frac{\partial}{\partial\gamma\partial\gamma^{\prime}} l(X_i,\hat{\gamma}) = -\frac{\Phi(X_i^{\prime}\gamma)\cdot[1-\Phi(X_i^{\prime}\gamma)]}{\left[\Phi(X_i^{\prime}\gamma)\cdot[1-\Phi(X_i^{\prime}\gamma)]\right]^2}\cdot \phi(X_i^{\prime}\gamma)^2\cdot X_i^{\prime}X_i = -\frac{\phi(X_i^{\prime}\gamma)^2}{\left[\Phi(X_i^{\prime}\gamma)\cdot[1-\Phi(X_i^{\prime}\gamma)]\right]}\cdot X_i^{\prime}X_i
-\frac{\partial}{\partial\gamma} l(X_i,\hat{\gamma})\cdot \frac{\partial}{\partial\gamma^{\prime}} l(X_i,\hat{\gamma})$

However, since allowing for heteroskedasticity implies the model is incorrect, I will simply calculate the asymptotic variance assuming correct specification and thus that the Fisher Information Equality holds.

Note the following: 

$\widehat{\mathsf{s.e}.}(\hat{\gamma}) = 
\mathsf{diag}\left(\left[\frac{1}{N}\widehat{\mathsf{AVar}}\left(\sqrt{N}(\hat{\gamma} - \gamma_o)\right)\right]^{\frac{1}{2}}\right)$

In [8]:
Xg = X*Optim.minimizer(MLE)
Phi = cdf(Normal(), Xg)
phi = pdf(Normal(), Xg)
val = zeros(k,k)
for i in 1:N
    val += ((((phi[i])^2) / (Phi[i]*(1-Phi[i])))*(X[i,:]*X[i,:]'))
end
avar = inv(val/N)
standerr = sqrt(diag(avar/N))
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
Julia = round(Optim.minimizer(MLE),3), 
J_SEs = round(standerr,3),
STATA = round(gmSTATA,3),
S_SEs = [0.031, 0.024, 0.018, 0.012, 0.008])

,vars,Julia,J_SEs,STATA,S_SEs
1,cons,-1.882,0.031,-1.88,0.031
2,edu,1.779,0.024,1.78,0.024
3,wht,0.273,0.018,0.27,0.018
4,msa,0.053,0.012,0.05,0.012
5,expr,0.291,0.008,0.29,0.008


Simulated Maximum Likelihood
============================

Now, we want to recalculate the above model, assuming that we cannot compute $\Phi()$. We will simulate this using three approaches. I will use the Crude Frequency simulator, an importance sampling simulator, and an antivariate crude frequency simulator.

For this, we must simulate the value of $\Phi(X_i^{\prime}\gamma)$ for each observation, thus $(N\cdot R)$ total simulations. 
We use the following simulators:
1. Crude Frequency
2. Importance Sampling
3. Antithetic Variate Crude Frequency

Crude Frequency
===============

In [9]:
function CrFr(Xg::Array{Float64,1})
    # This function is a non-general Crude Freq simulator for probit estimation.
    # Want Pr(X_i*g > - e_i) by sample avg of 1[X_i*g > - e_ir] for r = 1:R
    # Arguments: R simulations, Xg candidate X*gamma, gamma is coef vector
    nu = Array(Float64,N,R) 
    Icf = Array(Float64,N,1)
    # Get the Xg values greater than simulated errors, and convert logical to numeric
    nu = convert(Array{Float64,2},(Xg.>= (-1)*u1))
    # Average of the simulations for each observation
    Icf = vec((1/R)*sum(nu,2))
end

CrFr (generic function with 1 method)

In [10]:
############ Construct Likelihood Function
function f_SimlnLike1(gm::Vector)
    ## This creates a lnL function of given parameters for Sim.MLE
    # Preallocate Vectors
    t1 = Array(Float64,1,k)
    t2 = Array(Float64,N,1)
    t3 = Array(Float64,N,1)
    t4 = Array(Float64,N,1)
    t5 = Array(Float64,1,1)
    # Calculations
    t1 = [gm[1] gm[2] gm[3] gm[4] gm[5]]
    t2 = vec(X*t1')
    # CrFr Simulation of CDF of Standard Normal dist. eval. at each obs.
    t3 = CrFr(t2)
    # Column vector of lnL_i
    t4 = F.*log(t3) + (1 - F).*log(1-t3)
    # Sum of lnL_i = lnL; return the negation for minimization
    t5 = sum(t4)
    return (-1)*t5
end

f_SimlnLike1 (generic function with 1 method)

In [11]:
# Crude Freq at 100 Sim --- Takes about 3min
R = 100;
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
@time sMLE11 = optimize(f_SimlnLike1, gmOLS, opts)

 85

In [12]:
# Crude Freq at 500 Sim --- Takes about 10min
R = 500
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
@time sMLE12 = optimize(f_SimlnLike1, gmSTATA, opts)

481.

In [13]:
# Crude Freq at 1000 Sim ---- Takes about 20min!
R = 1000
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
@time sMLE13 = optimize(f_SimlnLike1, gmSTATA, opts)

889.139479 seconds (18.48 M allocations: 569.219 GB, 18.21% gc time)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.88,1.78,0.27,0.05,0.29]
 * Minimizer: [-1.8879350054142956,1.7728060857032728, ...]
 * Minimum: 2.828608e+04
 * Iterations: 183
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 197

In [14]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
OLS = round(gmOLS,3),
#MLE = round(Optim.minimizer(MLE),3), 
MLE = round(gmSTATA,3), 
CF100 = round(Optim.minimizer(sMLE11),3), 
CF500 = round(Optim.minimizer(sMLE12),3), 
CF1000 = round(Optim.minimizer(sMLE13),3))

,vars,OLS,MLE,CF100,CF500,CF1000
1,cons,-0.094,-1.88,-1.583,-1.891,-1.888
2,edu,0.568,1.78,1.603,1.78,1.773
3,wht,0.092,0.27,0.132,0.279,0.287
4,msa,0.016,0.05,0.069,0.052,0.054
5,expr,0.099,0.29,0.265,0.297,0.294


In [15]:
## AVar of Simulators
XG = X*Optim.minimizer(sMLE11);
R = 100;
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
CRD = CrFr(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (CRD[i]*(1-CRD[i])))*(X[i,:]*X[i,:]'))
end
avar11 = inv(val/N);
standerr11 = sqrt(diag(avar11/N));

## AVar of Simulators
XG = X*Optim.minimizer(sMLE12);
R = 500;
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
CRD = CrFr(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (CRD[i]*(1-CRD[i])))*(X[i,:]*X[i,:]'))
end
avar12 = inv(val/N);
standerr12 = sqrt(diag(avar12/N));

## AVar of Simulators
XG = X*Optim.minimizer(sMLE13);
R = 1000;
srand(537099);
# Draw R Normal simulations
u1 = rand(Normal(), N, R);
CRD = CrFr(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (CRD[i]*(1-CRD[i])))*(X[i,:]*X[i,:]'))
end
avar13 = inv(val/N);
standerr13 = sqrt(diag(avar13/N));

In [16]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
MLE = [0.031, 0.024, 0.018, 0.012, 0.008],
CF100 = round(standerr11,3), 
CF500 = round(standerr12,3), 
CF1000 = round(standerr13,3))

,vars,MLE,CF100,CF500,CF1000
1,cons,0.031,0.029,0.031,0.031
2,edu,0.024,0.022,0.024,0.024
3,wht,0.018,0.018,0.018,0.018
4,msa,0.012,0.012,0.012,0.012
5,expr,0.008,0.008,0.008,0.008


Importance Sampling
===================

$\Pr(D_i = 1|X_i) = \Pr(X_i^{\prime}\gamma \ge \epsilon_i) = \int 1\left[X_i^{\prime}\gamma \ge \epsilon_i\right] f(\epsilon)d\epsilon$

For importance sampling, draw $u_i \sim U(0,1)$ then find $\eta_i = G^{-1}(u_i)$.
Calculate $w(\eta_i) = \frac{f(\eta_i)}{g(\eta_i)}$ and the 
$\tilde{w}(\eta_i) = w(\eta_i)\ / \int w(\eta_i)di.$

Take the sumulator agverage: $\sum_{r=1:R} \left[1\left[X_i^{\prime}\gamma \ge - \eta_i \right]\cdot \tilde{w}(\eta_{ir})\right]$. 
This is refered to a normalized importance sampler.
Note this is different than the lecture notes to ensure that $\widehat{\Phi()} \in (0,1)$, and that the division by $R$ twice cancels out.

For this problem, using the standard normal and standard logistic, $w(\eta_i) = \frac{1}{\sqrt{2\pi}}\left[e^{\frac{-\eta_i^2}{2}}\right]\cdot\left[\frac{e^{-\eta_i}}{(1 + e^{-\eta_i})^{2}}\right]^{-1} = \phi(\eta_i)\cdot(1+e^{-\eta_i})^{2}$

In [17]:
function ImpSmp(Xg::Array{Float64,1})
    # This function is a non-general Importance Sampling simulator for probit estimation.
    # Using the density of the standard logistic distribution as our weighting function, g(u)
    # Want Pr(X_i*g > e_i) by sample avg of 1[X_i*g > - e_ir] /  for r = 1:R
    # Arguments: R simulations, Xg candidate X*gamma, gamma is coef vector
    nu2 = Array(Float64,N,R) 
    Iis = Array(Float64,N,1)
    # Get the Xg values greater than simulated errors, and convert logical to numeric
    nu2 = convert(Array{Float64,2},(Xg.>=(-1.0)*u2))
    wt2 = sum(nu2 .* WT,2)
    # Here the Normaized Impt Sampler rather than only divide by R
    Iis = wt2 ./ WTsum
end

ImpSmp (generic function with 1 method)

In [18]:
############ Construct Likelihood Function
function f_SimlnLike2(gm::Vector)
    ## This creates a lnL function of given parameters for Sim.MLE
    # (Re)Set Random Seed to Prevent "Chattering" - use same simulations each iteration    
    # Preallocate Vectors
    t1 = Array(Float64,1,k)
    t2 = Array(Float64,N,1)
    t3 = Array(Float64,N,1)
    t4 = Array(Float64,N,1)
    t5 = Array(Float64,1,1)
    # Calculations
    t1 = [gm[1] gm[2] gm[3] gm[4] gm[5]]
    t2 = vec(X*t1')
    # CrFr Simulation of CDF of Standard Normal dist. eval. at each obs.
    t3 = ImpSmp(t2)
    # Column vector of lnL_i
    t4 = F.*log(t3) + (1.0 - F).*log(1.0 - t3)
    # Sum of lnL_i = lnL; return the negation for minimization
    t5 = sum(t4)
    return (-1.0)*t5
end

f_SimlnLike2 (generic function with 1 method)

In [19]:
# Draw Uniform R simulations --- takes about 2min
R = 100;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
## Optimize!
@time sMLE21 = optimize(f_SimlnLike2, gmOLS, opts)

115.201729 seconds (65.56 M allocations: 98.700 GB, 16.27% gc time)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-0.09421108407456535,0.568001095931335, ...]
 * Minimizer: [-1.8595501398432197,1.7720276018281365, ...]
 * Minimum: 2.855221e+04
 * Iterations: 291
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 360

In [20]:
# Draw Uniform R simulations --- Takes about 13min
R = 500;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
## Optimize!
@time sMLE22 = optimize(f_SimlnLike2, gmSTATA, opts)

397.831502 seconds (42.05 M allocations: 268.248 GB, 9.34% gc time)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.88,1.78,0.27,0.05,0.29]
 * Minimizer: [-1.874530030640737,1.7731889854092202, ...]
 * Minimum: 2.833152e+04
 * Iterations: 137
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 150

In [21]:
# Draw Uniform R simulations
R = 1000;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
## Optimize!
@time sMLE23 = optimize(f_SimlnLike2, gmSTATA, opts)

930.232676 seconds (57.73 M allocations: 635.119 GB, 8.79% gc time)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.88,1.78,0.27,0.05,0.29]
 * Minimizer: [-1.8833411007783787,1.7762060442507785, ...]
 * Minimum: 2.829690e+04
 * Iterations: 154
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 164

In [22]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
OLS = round(gmOLS,3),
#MLE = round(Optim.minimizer(MLE),3), 
MLE = round(gmSTATA,3), 
IS100 = round(Optim.minimizer(sMLE21),3), 
IS500 = round(Optim.minimizer(sMLE22),3), 
IS1000 = round(Optim.minimizer(sMLE23),3))

,vars,OLS,MLE,IS100,IS500,IS1000
1,cons,-0.094,-1.88,-1.86,-1.875,-1.883
2,edu,0.568,1.78,1.772,1.773,1.776
3,wht,0.092,0.27,0.268,0.271,0.275
4,msa,0.016,0.05,0.063,0.055,0.056
5,expr,0.099,0.29,0.275,0.291,0.292


In [23]:
## AVar of Simulators
XG = X*Optim.minimizer(sMLE21);
R = 100;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
IMP = ImpSmp(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (IMP[i]*(1-IMP[i])))*(X[i,:]*X[i,:]'))
end
avar21 = inv(val/N);
standerr21 = sqrt(diag(avar21/N));

## AVar of Simulators
XG = X*Optim.minimizer(sMLE22);
R = 500;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
IMP = ImpSmp(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (IMP[i]*(1-IMP[i])))*(X[i,:]*X[i,:]'))
end
avar22 = inv(val/N);
standerr22 = sqrt(diag(avar22/N));

## AVar of Simulators
XG = X*Optim.minimizer(sMLE23);
R = 1000;
srand(537099);
u2 = rand(Logistic(), N, R);
WT = pdf(Normal(), u2) ./ pdf(Logistic(), u2);
WTsum = sum(WT,2);
IMP = ImpSmp(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (IMP[i]*(1-IMP[i])))*(X[i,:]*X[i,:]'))
end
avar23 = inv(val/N);
standerr23 = sqrt(diag(avar23/N));

In [24]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
MLE = [0.031, 0.024, 0.018, 0.012, 0.008],
IS100 = round(standerr21,3), 
IS500 = round(standerr22,3), 
IS1000 = round(standerr23,3))

,vars,MLE,IS100,IS500,IS1000
1,cons,0.031,0.03,0.031,0.031
2,edu,0.024,0.023,0.024,0.024
3,wht,0.018,0.018,0.018,0.018
4,msa,0.012,0.012,0.012,0.012
5,expr,0.008,0.008,0.008,0.008


Antivariate Crude Frequency
===========================

Similar to the Crude Frequency simulator, except we also calculate $\Phi^{-1}(1 - u_{ir})$ to get a negatively correlated random variable to contruct a second crude frequency simulator that we will average with the one constructed by $\Phi^{-1}(u_{ir})$.

In [25]:
function AVSim(Xg::Array{Float64,1})
    # This function is a non-general Importance Sampling simulator for probit estimation.
    # Using the density of the standard logistic distribution as our weighting function, g(u)
    # Want Pr(X_i*g > e_i) by sample avg of 1[X_i*g > - e_ir] /  for r = 1:R
    # Arguments: R simulations, Xg candidate X*gamma, gamma is coef vector
    r = Float64(R)
    nu31 = Array(Float64,N,R)
    nu32 = Array(Float64,N,R)
    IAV = Array(Float64,N,1)
    # Get the Xg values greater than simulated errors, and convert logical to numeric
    nu31 = convert(Array{Float64,2},(Xg.>=(-1.0)*u31))
    nu32 = convert(Array{Float64,2},(Xg.>=(-1.0)*u32))
    # Average of the simulations for each observation
    IAV = sum(nu31 + nu32, 2) / (2.0*r)
end

AVSim (generic function with 1 method)

In [26]:
############ Construct Likelihood Function
function f_SimlnLike3(gm::Vector)
    ## This creates a lnL function of given parameters for Sim.MLE
    # (Re)Set Random Seed to Prevent "Chattering" - use same simulations each iteration
    srand(537099)    
    # Preallocate Vectors
    t1 = Array(Float64,1,k)
    t2 = Array(Float64,N,1)
    t3 = Array(Float64,N,1)
    t4 = Array(Float64,N,1)
    t5 = Array(Float64,1,1)
    # Calculations
    t1 = [gm[1] gm[2] gm[3] gm[4] gm[5]]
    t2 = vec(X*t1')
    # CrFr Simulation of CDF of Standard Normal dist. eval. at each obs.
    t3 = AVSim(t2)
    # Column vector of lnL_i
    t4 = F.*log(t3) + (1 - F).*log(1-t3)
    # Sum of lnL_i = lnL; return the negation for minimization
    t5 = sum(t4)
    return (-1)*t5
end

f_SimlnLike3 (generic function with 1 method)

In [27]:
# Crude Freq at 100 Sim --- Takes about 5min
R = 100;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
@time sMLE31 = optimize(f_SimlnLike3, gmOLS, opts)

394.

In [28]:
# AV Crude Freq at 100 Sim --- Takes about 5min
R = 100;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
@time sMLE31 = optimize(f_SimlnLike3, gmOLS, opts)

395.319562 seconds (70.50 M allocations: 176.507 GB, 57.39% gc time)


In [29]:
# AV Crude Freq at 500 Sim --- Takes about 
R = 500;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
@time sMLE32 = optimize(f_SimlnLike3, gmSTATA, opts)

1077.217933 seconds (66.63 M allocations: 642.308 GB, 24.81% gc time)


In [30]:
# AV Crude Freq at 1000 Sim --- Takes about 
R = 1000;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
@time sMLE33 = optimize(f_SimlnLike3, gmSTATA, opts)

2244.818797 seconds (92.59 M allocations: 1.371 TB, 16.65% gc time)


Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [-1.88,1.78,0.27,0.05,0.29]
 * Minimizer: [-1.883698428814355,1.7832650464907938, ...]
 * Minimum: 2.828129e+04
 * Iterations: 187
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 210

In [31]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
OLS = round(gmOLS,3),
#MLE = round(Optim.minimizer(MLE),3), 
MLE = round(gmSTATA,3), 
AV100 = round(Optim.minimizer(sMLE31),3), 
AV500 = round(Optim.minimizer(sMLE32),3), 
AV1000 = round(Optim.minimizer(sMLE33),3))

,vars,OLS,MLE,AV100,AV500,AV1000
1,cons,-0.094,-1.88,-1.825,-1.874,-1.884
2,edu,0.568,1.78,1.747,1.785,1.783
3,wht,0.092,0.27,0.262,0.256,0.273
4,msa,0.016,0.05,0.041,0.044,0.044
5,expr,0.099,0.29,0.289,0.293,0.292


In [35]:
## AVar of Simulator - 1
XG = X*Optim.minimizer(sMLE31);
R = 100;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
AVS = AVSim(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (AVS[i]*(1-AVS[i])))*(X[i,:]*X[i,:]'))
end
avar31 = inv(val/N);
standerr31 = sqrt(diag(avar31/N));

## AVar of Simulator - 2
XG = X*Optim.minimizer(sMLE32);
R = 500;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
AVS = AVSim(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (AVS[i]*(1-AVS[i])))*(X[i,:]*X[i,:]'))
end
avar32 = inv(val/N);
standerr32 = sqrt(diag(avar32/N));

## AVar of Simulators
XG = X*Optim.minimizer(sMLE33);
R = 1000;
srand(537099);
# Draw R Normal simulations
u3 = rand(N, R);
u31 = norminvcdf.(u3);
u32 = norminvcdf.(1.0 - u3);
AVS = AVSim(XG);
phi = pdf(Normal(), XG);
val = zeros(k,k);
for i in 1:N
    val += ((((phi[i])^2) / (AVS[i]*(1-AVS[i])))*(X[i,:]*X[i,:]'))
end
avar33 = inv(val/N);
standerr33 = sqrt(diag(avar33/N));

In [36]:
DataFrame(vars = ["cons", "edu", "wht", "msa", "expr"],
MLE = [0.031, 0.024, 0.018, 0.012, 0.008],
AV100 = round(standerr31,3), 
AV500 = round(standerr32,3), 
AV1000 = round(standerr33,3))

,vars,MLE,AV100,AV500,AV1000
1,cons,0.031,0.03,0.031,0.031
2,edu,0.024,0.023,0.024,0.024
3,wht,0.018,0.018,0.018,0.018
4,msa,0.012,0.012,0.012,0.012
5,expr,0.008,0.008,0.008,0.008


In [34]:
Dates.format(now(), "HH:MM")

"16:04"